<a href="https://colab.research.google.com/github/nityanandkore/6162-KDD/blob/master/Homework10_TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!rm 6162-KDD
!git clone https://github.com/nityanandkore/6162-KDD.git

rm: cannot remove '6162-KDD': No such file or directory
Cloning into '6162-KDD'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 161 (delta 16), reused 0 (delta 0), pack-reused 129
Receiving objects: 100% (161/161), 26.38 MiB | 25.25 MiB/s, done.
Resolving deltas: 100% (82/82), done.


In [0]:
from zipfile import ZipFile

# Create a ZipFile Object and load sample.zip in it
with ZipFile('6162-KDD/DataSet/MovieReviews.zip', 'r') as zipObj:
# Extract all the contents of zip file in current directory
      zipObj.extractall('6162-KDD/DataSet/')

In [0]:
import os

cwd = os.getcwd()  # Get the current working directory (cwd)
dirpath=os.chdir("/content/6162-KDD/DataSet/MovieReviews/")
fileslist = os.listdir(dirpath)  # Get all the files in that directory
print("Files in %r: %s" % (dirpath, fileslist))

Files in None: ['18272.txt', '2110.txt', '17902.txt', '18068.txt', '2085.txt', '17753.txt', '1943.txt', '1858.txt', '17147.txt', '2094.txt', '17219.txt', '1928.txt', '1974.txt', '2081.txt', '17108.txt', '2098.txt', '2079.txt', '1912.txt', '1985.txt', '2095.txt', '17139.txt', '17460.txt', '2089.txt', '17341.txt', '17758.txt', '18227.txt', '17384.txt', '17119.txt', '17192.txt', '18032.txt', '2046.txt', '17501.txt', '17695.txt', '2031.txt', '17534.txt', '1990.txt', '1994.txt', '17111.txt', '1976.txt', '18307.txt', '1908.txt', '16748.txt', '17811.txt', '17761.txt', '17662.txt', '2076.txt', '18282.txt', '17254.txt', '18141.txt', '18067.txt', '2062.txt', '17303.txt', '17255.txt', '17757.txt', '18473.txt', '17243.txt', '17532.txt', '2043.txt', '17447.txt', '18413.txt', '17803.txt', '2045.txt', '18263.txt', '17150.txt', '2007.txt', '1967.txt', '18181.txt', '2115.txt', '18368.txt', '2005.txt', '1910.txt', '18161.txt', '1968.txt', '2051.txt', '1991.txt', '18136.txt', '1864.txt', '1865.txt', '183

In [0]:
import pandas as pd
import re
sub = 9999
df_list= []
#df = pd.DataFrame(columns=['Review','Writer'])
for file in fileslist:
    filecontent = open(file,'rb').read()
    #print(filecontent)
    flag= [int(i) for i in file.split('.') if i.isdigit()]
    #print(flag)
    for x in flag:
            if sub >= x:
                #print(x)
                df_list.append([filecontent, 0])
            if sub < x:
                 df_list.append([filecontent, 1])          
    
df_list


[[b'GODS AND MONSTERS (director:Bill Condon; cast: (James Whale) Ian McKellen, (Clayton Boone) Brendan Fraser, (Whale\'s housekeeper, Hanna) Lynn Redgrave, (Boone\'s girlfriend, Betty) Lolita Davidovich, (David Lewis) David Dukes, (Mr. Kay) Jack Plotinick, (Elsa Lanchester) Rosalind Ayres, (Boris Karloff) Jack Betts, 1998)\nJames Whale (Ian) is the son of working class parents in north England, who escaped his early poverty to become a successful director, noted for his famous monster films. James Whale directed the original FRANKENSTEIN (31), THE BRIDE OF FRANKENSTEIN (35), THE OLD DARK HOUSE (32), and THE INVISIBLE MAN (33)--- which was his personal favorite. Whale, always the aesthete and snob, comments to an interviewer that he just directed the first two Frankensteins, the others were done by hacks. He also did some non-horror films, most notable is the classy musical SHOWBOAT (36), and his most inauspicious one is, THE ROAD BACK (37).\nHe died of mysterious circumstances in his H

In [0]:
type(df_list)

list

In [0]:
len(df_list)

180

In [0]:
df = pd.DataFrame(df_list, columns =['Review','Writer']) 
df 

,Review,Writer
0,b'GODS AND MONSTERS (director:Bill Condon; cas...,1
1,"b'Starring: Suzy Amis, Bo Hopkins, Ian McKelle...",0
2,b'CHAIN LETTERS (director: Mark Rappaport; cas...,1
3,b'IDIOTERNE (director: Lars von Trier; cast: B...,1
4,b'In the quiet New England town of Castle Rock...,0
...,...,...
175,"b'Starring: Sam Neill, Jeff Goldblum, Richard ...",0
176,"b""On the rebound from a lifeless seven-year ma...",0
177,b'Date Released: 6/11/93; wide release 6/18/93...,0
178,b'BELOVED (director: Jonathan Demme; cast: Opr...,1


In [0]:
df.Writer.unique()

array([1, 0])

In [0]:
df['Writer'].value_counts()

1    100
0     80
Name: Writer, dtype: int64

In [0]:
# This is to vectorize the text corpus. After these codes, the X object will be the input vector for machine learning models.
# When transform into vectors, we do NOT use the raw count of a word in a document. Instead, we use the word's tf-idf score in a document.
# max_df=0.5 means ingoring words that appear in more than 50% of the documents; min_df=2 means ignoring words that appear in less than 2 documents.

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english', use_idf=True,encoding='latin-1')

X = vectorizer.fit_transform(df.Review)

In [0]:
#X in the previuos step is hidimentional data, we need to use some dimentionality reduction technique. In this case, we will use SVD (Singular Value Decomposition), which is a common matrix decomposition technique.
#We want to reduce the dimentionality to 5.
#We have to re-normalize after we run our SVD on the dataset.
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer

n_components = 3
svd = TruncatedSVD(n_components)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X = lsa.fit_transform(X)

In [0]:
X

array([[ 9.54594820e-01, -1.77785750e-01, -2.39041745e-01],
       [ 8.68997869e-01,  4.77951196e-01, -1.28083401e-01],
       [ 2.36648729e-01, -2.20815996e-01,  9.46170003e-01],
       [ 8.22164336e-01, -4.65188413e-01, -3.28093805e-01],
       [ 9.03423683e-01,  4.10699486e-01, -1.23091760e-01],
       [ 9.83978743e-01, -6.93459139e-02, -1.64246699e-01],
       [ 9.32051357e-01,  1.91380024e-01, -3.07658826e-01],
       [ 9.32446613e-01,  3.59949452e-01, -3.13002429e-02],
       [ 9.00921064e-01, -3.42842697e-01, -2.66082923e-01],
       [ 8.99606624e-01,  4.36287648e-01, -1.90002823e-02],
       [ 9.72286362e-01, -2.28417789e-01,  4.98452009e-02],
       [ 8.75272908e-01,  4.33636502e-01,  2.14141824e-01],
       [ 8.58845626e-01,  5.12098377e-01,  1.18085709e-02],
       [ 8.58652208e-01,  4.94684096e-01,  1.34179096e-01],
       [ 8.78616780e-01, -4.67544792e-01,  9.71309481e-02],
       [ 8.17236669e-01,  5.68741644e-01,  9.30439091e-02],
       [ 8.50001177e-01,  5.22922915e-01

In [0]:
X.shape

(180, 3)

In [0]:
#Randomly select 80% (180*80% = 144) rows from X as the training set
import numpy as np
training_idx = np.random.choice(X.shape[0], size=144, replace=False)
X_training = X[training_idx, :]

In [0]:
#The remaining is the test set
test_idx = list(set(range(X.shape[0])) - set(training_idx))
X_test = X[test_idx, :]

In [0]:
labels = df.Writer

In [0]:
labels

0      1
1      0
2      1
3      1
4      0
      ..
175    0
176    0
177    0
178    1
179    1
Name: Writer, Length: 180, dtype: int64

In [0]:
#The same split for the labels list
labels_training = [labels[i] for i in training_idx]
labels_test = [labels[j] for j in test_idx]

In [0]:
#This is to use Naive Bayes Classifier to predict the label of each news article.
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_training, labels_training)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
#This is the prediction result.
gnb.predict(X_test)

array([1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1])

In [0]:
#This is the confusion matrix.
import sklearn.metrics
sklearn.metrics.confusion_matrix(gnb.predict(X_test),labels_test)

array([[15,  0],
       [ 2, 19]])

In [0]:
#This is the accuracy score.
sklearn.metrics.accuracy_score(gnb.predict(X_test),labels_test)

0.9444444444444444